## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же класса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [0]:
import gensim
import urllib.request

In [2]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x7f864861ef28>)

In [3]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [5]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(word, ': ', sim)
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S :  0.7165195941925049
месяц_S :  0.631048858165741
вечер_S :  0.5828739404678345
утро_S :  0.5676207542419434
час_S :  0.5605547428131104
минута_S :  0.5297019481658936
гекатомбеон_S :  0.4897990822792053
денек_S :  0.48224714398384094
полчаса_S :  0.48217129707336426
ночь_S :  0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S :  0.6946247816085815
утро_S :  0.57301926612854
ноченька_S :  0.5582467317581177
рассвет_S :  0.5553582906723022
ночка_S :  0.5351512432098389
полдень_S :  0.5334426164627075
полночь_S :  0.478694349527359
день_S :  0.4780748784542084
сумерки_S :  0.4390218257904053
фундерфун_S :  0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681 

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Находим косинусную близость пары слов:

In [9]:
print(model_ru.similarity('человек_S', 'кошка_S'))

0.1664258


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [12]:
model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'], topn=1)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('пельмень_S', 0.3977898955345154)]

In [13]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'ананас_S'

## Загрузим word2vec модель без меток частей речи

In [14]:
!wget http://vectors.nlpl.eu/repository/11/187.zip

--2020-03-28 09:51:29--  http://vectors.nlpl.eu/repository/11/187.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2692389519 (2.5G) [application/zip]
Saving to: ‘187.zip’

187.zip             100%[===================>]   2.51G  9.70MB/s    in 4m 42s  

2020-03-28 09:56:12 (9.11 MB/s) - ‘187.zip’ saved [2692389519/2692389519]



In [15]:
!unzip 187.zip

Archive:  187.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [0]:
from gensim.models.keyedvectors import FastTextKeyedVectors

In [17]:
m = FastTextKeyedVectors.load('model.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Посмотрим размер словаря:

In [18]:
len(m.vocab)

192415

In [19]:
m['исправляй'][:20] # первые 20 чисел вектора

array([-1.3566604 , -1.3532224 ,  0.9593475 , -1.8901982 , -0.59628433,
       -0.68555176,  2.905148  ,  1.9639157 , -0.32325912, -0.32197556,
        0.94435304,  2.501834  ,  0.40444824, -0.4573581 , -1.4188358 ,
       -1.8840065 , -1.5600121 , -0.5360782 , -0.1541971 , -0.702145  ],
      dtype=float32)

In [20]:
len(m['исправляй'])

300

Найти близкие:

In [21]:
m.most_similar('спорт')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('спортинг', 0.8338270783424377),
 (',спорт', 0.8087105751037598),
 ('р-спорт', 0.795436441898346),
 ('спортлото', 0.7781082987785339),
 ('спорти', 0.7661596536636353),
 ('велоспорт', 0.7568544149398804),
 ('спортшкол', 0.7558624744415283),
 ('спортклуб', 0.7546058893203735),
 ('спорт-бар', 0.7508780360221863),
 ('спортивка', 0.7459163665771484)]

In [33]:
m.similarity('кот', 'котик')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


0.73086005

Посчитать близость самостоятельно:

In [0]:
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
sim_matrix = cosine_similarity([m['кот'], m['собака']])
sim_matrix[0][1] # получаем марицу близостей

0.6739322

In [30]:
sim_matrix[0][1]

0.6739322

In [31]:
cosine_similarity([m['кот']], [m['котик']])[0][0]

0.73086

In [32]:
cosine_similarity([m['кот']], [m['крот']])[0][0]

0.5116138

Сложить вектора:

In [0]:
something = m['москва'] + m['омск'] + m['калуга']

In [42]:
cosine_similarity([something], [m['рязань']])

array([[0.7582565]], dtype=float32)

### Word2vec в реальных задачах

**Задание**: напишите функцию, которая принимает текст, а возвращает эмбеддинг этого текста, основанный на словах, входящих в состав.

In [0]:
from nltk.tokenize import word_tokenize

In [50]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
text1 = 'президент россии прилетел в америку'
text2 = 'путин прибыл в сша'

In [47]:
len(sum([m['омск'], m['калуга']]))

300

In [0]:
def embed_text(text):
  tokens = word_tokenize(text)
  word_embeddings = []
  for token in tokens:
    try:
      word_embeddings.append(m[token])
    except AttributeError:
      print('problem with word', token)

  text_embedding = sum(word_embeddings) / len(word_embeddings)
  return text_embedding

In [74]:
result1 = embed_text(text1)
result2 = embed_text(text2)
# result3 = embed_text(text3)


problem with word прилетел
problem with word в
problem with word прибыл
problem with word в


In [75]:
cosine_similarity([result1], [result2])

array([[0.86072224]], dtype=float32)

In [70]:
cosine_similarity([result2], [result3])

array([[0.8247774]], dtype=float32)

### А теперь давайте применим это к задаче из прошлого семинара

Те данные: https://drive.google.com/open?id=1eDq0dDWM7601IPbr0lAbO3n9qZC1UYeO

In [0]:
import pandas as pd

In [0]:
# для колаба:
# 1. запускаете эту ячейку
# 2. ждёте
# 3. внизу появляется кнопка с загрузкой с компьютера
# 4. выбираете файл на компьютере и загружаете
# 5. доооолго ждёте, пока он скачается -_-
from google.colab import files
uploaded = files.upload()

Saving news_lenta_cropped.csv to news_lenta_cropped.csv


In [0]:
df = pd.read_csv('news_lenta_cropped.csv')

## Оценка word2vec моделей

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния до слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификаторов мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии: